# 객관식 문제 변형(251111)

In [2]:
import os, json
from tools.pipeline.config import ONEDRIVE_PATH

with open('multiple_classify.json', 'r', encoding='utf-8') as f:
    multiple = json.load(f)

## 모두고르시오

### 변형하기

In [5]:
from tools.core.llm_query import LLMQuery
llm = LLMQuery()

system_prompt = """당신은 프롬프트 전문가입니다. 다음은 ㄱ,ㄴ,ㄷ,ㄹ / 가,나,다,라 등 보기 중 옳은 것을 모두 고르는 문제입니다. 다음 문제의 보기를 찾아 선택지로 바꾸고 (예: ① ㄱ, ② ㄴ, ③ ㄷ, ④ ㄹ / ① 가, ② 나, ③ 다, ④ 라)
주어진 문제와 정답, 해설을 참고하여 선택지 중 옳은 것을 모두 고르는 문제로 변형하려고 합니다. 아래와 같은 프롬프트를 사용하려고 할 때, 올바른 프롬프트인지 확인해주세요."""
user_prompt = """"========================= 아래 ==========================
당신은 25년 경력의 문제 출제 전문가입니다.

변형 규칙
1) '모두 고르시오' 문제의 대상(보기)를 선택지로 바꿉니다(예: ① ㄱ, ② ㄴ, ③ ㄷ, ④ ㄹ / ① 가, ② 나, ③ 다, ④ 라).
2) 기존의 정답을 참고하여 옳은 선택지를 모두 고릅니다. (예: 기존 정답이 ㄱ,ㄷ 이면 새로운 정답은 ①,③).
3)그 외 문장과 LaTeX 수식이나 테이블 표현은 원형을 최대한 보존합니다.

  출력 형식
  {{
    "question_id": "문제번호",
    "question": "문제",
    "options": ["① 선택지1", "② 선택지2", "③ 선택지3", "④ 선택지4", "⑤ 선택지5"],
    "answer": ["정답번호1", "정답번호2", ...], 
    "explanation": "① 옳다(원래 답 중 하나): 근거 ... / ③ 옳다(원래 답 중 하나): 근거 ... / ⑤ 옳지 않다: 근거 ..."
  }}"""

response = llm.query_openrouter(system_prompt, user_prompt, model_name='openai/gpt-5')

In [7]:
print(response)

진단: 전반적으로 의도는 명확하고 유효합니다. 다만 실사용 시 혼동이나 오류를 줄이기 위해 몇 가지 보완이 필요합니다.

핵심 개선 포인트
- 옵션 개수 가변화: options를 항상 5개로 고정하지 말고, 보기(ㄱ/가 등)의 실제 개수만큼 생성하도록 명시하세요.
- 라벨-옵션 매핑 명확화: 원문 보기의 순서를 그대로 ①, ②, ③…에 일대일 대응시키는 규칙을 명시하세요.
- 질문 본문 보존: 보기(ㄱ/ㄴ/ㄷ/ㄹ 등)에 해당하는 문장들은 question 필드에 원문 그대로 남겨두고, options에는 라벨만 나열한다는 점을 분명히 하세요.
- 출력 엄격화: 출력은 오직 JSON만, 불필요한 설명/코드블록/주석 없이. 유효한 JSON 형식(따옴표, 쉼표, 줄바꿈 이스케이프) 준수도 명시하세요.
- 다양한 라벨 지원: ㄱ/ㄴ/ㄷ/ㄹ뿐 아니라 가/나/다/라, A/B/C/D 등 라벨 변형도 동일한 규칙으로 처리한다고 명시하세요.
- 무응답/전부 정답 처리: 정답이 없음(예: “해당 없음”)이거나 전부 정답인 경우 answer 배열을 [] 또는 모든 번호로 처리하는 규칙을 명시하세요.
- 해설 작성 지침: 각 옵션마다 옳다/옳지 않다와 근거를 1문장 이상으로 제시하도록 명시하세요(가능하면 제공된 해설의 핵심 문구를 요약/인용).  
- 예외 처리: 원문 정답과 해설이 상충할 경우 우선순위(보통 정답 우선)를 명시하고, 불가피하면 설명에 상충 사실을 간단히 표기.

개선된 프롬프트 제안문
당신은 25년 경력의 문제 출제 전문가입니다.

작업 목표
- ‘모두 고르시오’ 유형에서 보기(ㄱ/ㄴ/ㄷ/ㄹ, 가/나/다/라, A/B/C/D 등)를 선택지로 전환하고, 기존 정답을 그대로 매핑하여 정답 번호를 산출합니다.
- 원문 문장, 수식(LaTeX), 표는 가능한 한 그대로 보존합니다.

변형 규칙
1) 보기 라벨 식별: 문제 본문에 제시된 보기(예: ㄱ, ㄴ, ㄷ, ㄹ 또는 가, 나, 다, 라 등)를 원문 순서대로 식별합니다. 보기의 내용(문장)은 question 필드에 그대로 

In [8]:
pick_abcd = [p for p in multiple if p['pick'] == 'abcd']

In [12]:
from tools.core.llm_query import LLMQuery
import time
import logging
from datetime import datetime

# 로깅 설정
log_dir = 'logs'
os.makedirs(log_dir, exist_ok=True)
log_file = os.path.join(log_dir, f'multiple_answer_abcd.log')

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(log_file, encoding='utf-8'),
        logging.StreamHandler()
    ]
)

logger = logging.getLogger(__name__)

llm = LLMQuery()
output_dir = os.path.join('7_multiple_rw/pick_abcd')

logger.info(f"로그 파일: {log_file}")
logger.info("=" * 80)

logger.info(f"\n처리 시작")
parsed_responses = []
not_parsed_responses = []
no_responses = []

for idx, p in enumerate(pick_abcd, 1):
  question_id = p['file_id'] + '_' + p['tag']
  logger.info(f"\n{idx}/{len(pick_abcd)} - 문제 ID: {question_id} 처리 중...")
  # options_ct = len(p['options'])

  system_prompt = f"""당신은 25년 경력의 문제 출제 전문가입니다.

작업 목표
- ‘모두 고르시오’ 유형에서 보기(ㄱ/ㄴ/ㄷ/ㄹ, 가/나/다/라, A/B/C/D 등)를 선택지로 전환하고, 기존 정답을 그대로 매핑하여 정답 번호를 산출합니다.
- 원문 문장, 수식(LaTeX), 표는 가능한 한 그대로 보존합니다.

변형 규칙
1) 보기 라벨 식별: 문제 본문에 제시된 보기(예: ㄱ, ㄴ, ㄷ, ㄹ 또는 가, 나, 다, 라 등)를 원문 순서대로 식별합니다. 보기의 내용(문장)은 question 필드에 그대로 남깁니다.
2) 선택지 구성: options에는 라벨만을 표시합니다. 예: ① ㄱ, ② ㄴ, ③ ㄷ, ④ ㄹ. 보기 수에 맞춰 ①부터 순차적으로 생성합니다(개수 가변).
3) 정답 매핑: 기존 정답의 라벨(예: ㄱ, ㄷ)을 해당 번호(예: ①, ③)로 변환하여 answer 배열에 문자열로 담습니다. 정답이 없으면 [], 전부 정답이면 모든 번호를 포함합니다.
4) 해설: 각 번호(①, ②, …)마다 “옳다/옳지 않다”를 명시하고, 제공된 해설을 근거로 1문장 이상 요약 근거를 제시합니다. 예: “① 옳다(원래 답 중 하나): … / ② 옳지 않다: …”
5) 보존 원칙: 원문의 기타 문장, 수식(LaTeX), 표, 줄바꿈을 가능한 한 그대로 유지합니다. 내용 변형, 임의 추론은 금지합니다.
6) 출력 형식: 오직 유효한 JSON만 출력합니다. 코드블록, 추가 설명, 주석 금지.

출력 형식
{{
  "question_id": "문제번호",
  "question": "문제 원문(보기 문장 포함, LaTeX/표/줄바꿈 보존)",
  "options": ["① ㄱ", "② ㄴ", "③ ㄷ", "④ ㄹ"],
  "answer": ["①","③"],
  "explanation": "① 옳다(원래 답 중 하나): 근거 … / ② 옳지 않다: 근거 … / ③ 옳다(원래 답 중 하나): 근거 … / ④ 옳지 않다: 근거 …"
}}

추가 권고
- 라벨이 가/나/다/라 등인 경우에도 options는 ① 가, ② 나처럼 그대로 표기합니다.
- 보기 수가 9개를 초과할 경우, ①~⑨ 이후에는 ⑩, ⑪… 또는 일반 숫자 10), 11)로 일관되게 표기하는 규칙을 사전에 정해 두세요.
- 원문에 보기가 명시적 라벨 없이 줄글로만 제시되면 변환을 중단하고 에러를 출력하도록 별도 규칙을 두는 것이 안전합니다."""
      
   
  user_prompt = f"""
========== 다음 ===========
"""
  question_id = p['file_id'] + '_' + p['tag']
  question = p['question']
  options = p['options']
  answer = p['answer']
  explanation = p['explanation']

  user_prompt += f"""문제번호: {question_id}
문제: {question}
선택지: {options}
답: {answer}
해설: {explanation}
"""
  logger.info(f"  - API 호출 중... (모델: openai/o3)")
  time.sleep(0.6)
  try:
    response = llm.query_openrouter(system_prompt, user_prompt, model_name='openai/o3')
    logger.info(f"  - API 응답 수신 완료 (길이: {len(response) if response else 0}자)")
    try:
      # 단일 JSON 객체 파싱 시도
      parsed_response = None
      # 먼저 배열 형태로 시도
      parsed_response = llm.parse_api_response(response)
      
      # 배열이 아니면 단일 객체로 시도
      if parsed_response is None:
        try:
          # JSON 객체 찾기
          start_idx = response.find('{')
          end_idx = response.rfind('}') + 1
          
          if start_idx != -1 and end_idx > 0:
            json_str = response[start_idx:end_idx]
            parsed_response = json.loads(json_str)
            logger.info(f"  - 단일 JSON 객체 파싱 성공")
        except Exception as parse_err:
          logger.warning(f"  - JSON 객체 파싱 실패: {parse_err}")
          pass
      
      if parsed_response is not None:
        parsed_responses.append(parsed_response)
        logger.info(f"  ✓ 파싱 성공 - result.json에 저장")

        # 기존 파일이 있으면 읽어서 누적
        result_file = f'{output_dir}/result.json'
        os.makedirs(os.path.dirname(result_file), exist_ok=True)
        
        existing_results = []
        if os.path.exists(result_file):
          try:
            with open(result_file, 'r', encoding='utf-8') as f:
              existing_data = json.load(f)
              # 기존 데이터가 리스트면 그대로 사용, 아니면 리스트로 변환
              if isinstance(existing_data, list):
                existing_results = existing_data
              else:
                existing_results = [existing_data]
          except:
            existing_results = []
        
        # 새 결과 추가
        existing_results.append(parsed_response)
        
        # 누적된 결과 저장
        with open(result_file, 'w', encoding='utf-8') as f:
          json.dump(existing_results, f, ensure_ascii=False, indent=4)
        logger.info(f"  ✓ 저장 완료 (총 {len(existing_results)}개 결과 누적)")
      else:
        # 파싱 실패
        not_parsed_responses.append((p, response))
        logger.warning(f"  ✗ 파싱 실패 - not_parsed.json에 저장")

        # 기존 파일이 있으면 읽어서 누적
        not_parsed_file = f'{output_dir}/not_parsed.json'
        os.makedirs(os.path.dirname(not_parsed_file), exist_ok=True)
        
        existing_not_parsed = []
        if os.path.exists(not_parsed_file):
          try:
            with open(not_parsed_file, 'r', encoding='utf-8') as f:
              existing_not_parsed = json.load(f)
          except:
            existing_not_parsed = []
        
        # 새 결과 추가
        existing_not_parsed.append((p, response))
        
        # 누적된 결과 저장
        with open(not_parsed_file, 'w', encoding='utf-8') as f:
          json.dump(existing_not_parsed, f, ensure_ascii=False, indent=4)
        logger.warning(f"  ✓ 저장 완료 (총 {len(existing_not_parsed)}개 파싱 실패 누적)")

    except Exception as e:
      # 파싱 실패
      not_parsed_responses.append((p, response))
      logger.error(f"  ✗ 파싱 중 예외 발생: {str(e)}")

      # 기존 파일이 있으면 읽어서 누적
      not_parsed_file = f'{output_dir}/not_parsed.json'
      os.makedirs(os.path.dirname(not_parsed_file), exist_ok=True)
      
      existing_not_parsed = []
      if os.path.exists(not_parsed_file):
        try:
          with open(not_parsed_file, 'r', encoding='utf-8') as f:
            existing_not_parsed = json.load(f)
        except:
          existing_not_parsed = []
      
      # 새 결과 추가
      existing_not_parsed.append((p, response))
      
      # 누적된 결과 저장
      with open(not_parsed_file, 'w', encoding='utf-8') as f:
        json.dump(existing_not_parsed, f, ensure_ascii=False, indent=4)
      logger.warning(f"  ✓ 저장 완료 (총 {len(existing_not_parsed)}개 파싱 실패 누적)")

  except Exception as e:
    # 응답 없음
    response = None
    no_responses.append(p)
    logger.error(f"  ✗ API 호출 실패: {str(e)}")

    # 기존 파일이 있으면 읽어서 누적
    no_response_file = f'{output_dir}/no_response.json'
    os.makedirs(os.path.dirname(no_response_file), exist_ok=True)
    
    existing_no_responses = []
    if os.path.exists(no_response_file):
      try:
        with open(no_response_file, 'r', encoding='utf-8') as f:
          existing_no_responses = json.load(f)
      except:
        existing_no_responses = []
    
    # 새 결과 추가
    existing_no_responses.append(p)
    
    # 누적된 결과 저장
    with open(no_response_file, 'w', encoding='utf-8') as f:
      json.dump(existing_no_responses, f, ensure_ascii=False, indent=4)
    logger.warning(f"  ✓ 저장 완료 (총 {len(existing_no_responses)}개 응답 없음 누적)")
  
  logger.info(f"  {idx}/{len(pick_abcd)} - 문제 ID: {question_id} 처리 완료")
  # break

logger.info(f"\n 처리 완료 - 성공: {len(parsed_responses)}, 파싱실패: {len(not_parsed_responses)}, 응답없음: {len(no_responses)}")
logger.info("=" * 80)
  # break

2025-11-15 14:36:42,961 - INFO - 로그 파일: logs\multiple_answer_abcd.log
2025-11-15 14:36:42,962 - INFO - ================================================================================
2025-11-15 14:36:42,962 - INFO - 
처리 시작
2025-11-15 14:36:42,965 - INFO - 
1/164 - 문제 ID: SS0244_q_0578_0001 처리 중...
2025-11-15 14:36:42,966 - INFO -   - API 호출 중... (모델: openai/o3)
2025-11-15 14:36:44,316 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-15 14:36:52,719 - INFO -   - API 응답 수신 완료 (길이: 468자)
2025-11-15 14:36:52,720 - INFO -   - 단일 JSON 객체 파싱 성공
2025-11-15 14:36:52,720 - INFO -   ✓ 파싱 성공 - result.json에 저장
2025-11-15 14:36:52,722 - INFO -   ✓ 저장 완료 (총 1개 결과 누적)
2025-11-15 14:36:52,724 - INFO -   1/164 - 문제 ID: SS0244_q_0578_0001 처리 완료
2025-11-15 14:36:52,724 - INFO - 
2/164 - 문제 ID: SS0184_q_0303_0001 처리 중...
2025-11-15 14:36:52,725 - INFO -   - API 호출 중... (모델: openai/o3)
2025-11-15 14:36:53,614 - INFO - HTTP Request: POST https://openrouter.

In [ ]:
# 5문제: 1분 10초